In [1]:
from __future__ import print_function, division
import os, sys
import h5py
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

%matplotlib inline
import seaborn as sns
sns.set(context='talk', style='whitegrid')

In [2]:
import gctx_utils
import geode

## Load gene_meta, sig_meta and inst_meta

In [3]:
gene_info_df = pd.read_csv('data/GSE92742_Broad_LINCS_gene_info.txt.gz', 
                           compression='gzip',
                          sep='\t')
gene_info_df.set_index(gene_info_df.columns[0], inplace=True)
gene_info_df.index = gene_info_df.index.astype(str)
print(gene_info_df.shape)


(12328, 4)


In [4]:
LM_GENE_IDS = gene_info_df.query('pr_is_lm == 1').index
print(len(LM_GENE_IDS))

978


In [5]:
inst_info_df = pd.read_csv('data/GSE92742_Broad_LINCS_inst_info.txt.gz', 
                          compression='gzip',
                          sep='\t')
inst_info_df.set_index(inst_info_df.columns[0], inplace=True)
print(inst_info_df.shape)

(1319138, 10)


/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Keep trt_cp and ctl_vehicle
inst_info_df = inst_info_df.loc[inst_info_df['pert_type'].isin(['trt_cp', 'ctl_vehicle'])]\
    .query('cell_id == "PC3"')
# keep PC3 
print(inst_info_df.shape)

(96014, 10)


In [7]:
# Extract batch and det_plate from the instance_id
inst_info_df['batch'] = inst_info_df['rna_plate'].map(lambda x: '_'.join(x.split('_')[:3]))
inst_info_df['det_plate'] = inst_info_df.index.map(lambda x: x.split(':')[0])

In [8]:
sig_info_df = pd.read_csv('data/GSE92742_Broad_LINCS_sig_info.txt.gz', 
                          compression='gzip',
                          sep='\t')
sig_info_df = sig_info_df.set_index('sig_id')
sig_info_df.shape

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(473647, 11)

In [9]:
sig_info_df = sig_info_df.query('cell_id == "PC3" & pert_type == "trt_cp"')
print(sig_info_df.shape)
sig_info_df.head()

(23123, 11)


,pert_id,pert_iname,pert_type,cell_id,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_time_unit,pert_itime,distil_id
sig_id,,,,,,,,,,,
AML001_PC3_6H:BRD-A03772856:0.37037,BRD-A03772856,BRD-A03772856,trt_cp,PC3,0.37037,µM,500 nM,6,h,6 h,AML001_PC3_6H_X1_F1B10:J16|AML001_PC3_6H_X2_F1...
AML001_PC3_6H:BRD-A03772856:1.11111,BRD-A03772856,BRD-A03772856,trt_cp,PC3,1.11111,µM,1 µM,6,h,6 h,AML001_PC3_6H_X1_F1B10:J15|AML001_PC3_6H_X2_F1...
AML001_PC3_6H:BRD-A03772856:10,BRD-A03772856,BRD-A03772856,trt_cp,PC3,10,µM,10 µM,6,h,6 h,AML001_PC3_6H_X1_F1B10:I15|AML001_PC3_6H_X2_F1...
AML001_PC3_6H:BRD-A03772856:3.33333,BRD-A03772856,BRD-A03772856,trt_cp,PC3,3.33333,µM,3 µM,6,h,6 h,AML001_PC3_6H_X1_F1B10:I16|AML001_PC3_6H_X2_F1...
AML001_PC3_6H:BRD-A19037878:0.37037,BRD-A19037878,trichostatin-a,trt_cp,PC3,0.37037,µM,500 nM,6,h,6 h,AML001_PC3_6H_X1_F1B10:F18|AML001_PC3_6H_X2_F1...


In [10]:
sig_info_df['batch'] = sig_info_df.index.map(lambda x: x.split(':')[0])
sig_info_df['n_distil_ids'] = sig_info_df['distil_id'].map(lambda x: len(x.split('|')))
sig_info_df = sig_info_df.query('n_distil_ids > 1')
sig_info_df.shape

(22664, 13)

In [11]:
sig_info_df['batch'].value_counts()

CPC019_PC3_24H      364
CPC008_PC3_6H       364
CPC001_PC3_6H       364
CPC009_PC3_24H      364
CPC003_PC3_6H       364
CPC012_PC3_24H      364
CPC003_PC3_24H      364
CPC013_PC3_6H       364
CPC002_PC3_6H       364
CPD001_PC3_24H      364
CPC012_PC3_6H       364
CPC009_PC3_6H       364
CPC004_PC3_24H      364
CPC004_PC3_6H       364
CPC005_PC3_24H      363
CPC001_PC3_24H      363
CPC013_PC3_24H      363
CPC005_PC3_6H       363
CPC006_PC3_6H       363
CPC019_PC3_6H       363
CPC002_PC3_24H      363
CPC008_PC3_24H      362
CPD002_PC3_24H      361
CPD002_PC3_6H       361
CPD003_PC3_6H       360
PCLB003_PC3_24H     360
CPC006_PC3_24H      359
CPC007_PC3_6H       356
CPC007_PC3_24H      356
CPC010_PC3_6H       356
                   ... 
DOS056_PC3_24H      317
DOS047_PC3_24H      317
DOS046_PC3_24H      317
ERG012_PC3_24H      317
DOS054_PC3_24H      315
DOS034_PC3_24H      315
DOS051_PC3_24H      315
DOS055_PC3_24H      314
DOS033_PC3_24H      314
DOS050_PC3_24H      311
DOS058_PC3_24H  

In [12]:
gctx3 = h5py.File('data/GSE92742_Broad_LINCS_Level3_INF_mlr12k_n1319138x12328.gctx', 'r')

The normalization method is described in [Iskar et al PLoS comp Bio, 2010](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1000925):

```python
for batch in batches:
    1. discard control samples
    2. mean center each probe across treatment samples in batch
    3. average replicate treatment samples as the signature
```

<img src="img/mean_center_method.png" style="width: 400px;"/>


In [13]:
all_batches = sig_info_df['batch'].unique()
n_batches = len(all_batches)
print(n_batches)

75


In [14]:
!mkdir -p data/results

In [15]:
# Create gctx (h5 files) for the three types of signatures
shape = (len(LM_GENE_IDS), sig_info_df.shape[0])

gctx5_files = {
    'CD_center_LM': h5py.File('data/results/CD_center_level5_PC3_n%dx%d.gctx' % shape, 'w'),
    'avg_center_LM': h5py.File('data/results/Avg_center_level5_PC3_n%dx%d.gctx' % shape, 'w'),
}

# Write rids and cids
for gctx5_file in gctx5_files.values():
    cids = gctx5_file.create_dataset('/0/META/COL/id', 
                                     data=sig_info_df.index.values.astype(np.str))
    rids = gctx5_file.create_dataset('/0/META/ROW/id', 
                                     data=LM_GENE_IDS.values.astype(np.str))

gctx5_mats = {}
# Create dataset for the matrix inside each gctx file
for sig_name, gctx5_file in gctx5_files.items():
    mat = gctx5_file.create_dataset('/0/DATA/0/matrix', shape=shape, dtype=np.float32)
    gctx5_mats[sig_name] = mat

In [16]:
j = 0
for c, batch in enumerate(all_batches):
    sig_info_df_sub = sig_info_df.query('batch == "%s"' % batch)
    
    # Find all the treatment samples in this batch
    inst_info_df_sub = inst_info_df.query('batch == "%s" & pert_type == "trt_cp"' % batch)
    distil_ids_sub = inst_info_df_sub.index.tolist()
    
    print('===== Batch %d/%d %s =====' % (c, n_batches, batch))
    print('Number of signatures: %d\nNumber of pert samples: %d' % (sig_info_df_sub.shape[0], len(distil_ids_sub)))

    # Slice the matrix
    mat = gctx_utils.slice_matrix(gctx3, distil_ids_sub, LM_GENE_IDS)
    print(mat.shape)
    
    # Mean center the probes by batch
    mat_centered = gctx_utils.mean_center(mat, inst_info_df_sub['batch'])
    
    docs = Parallel(n_jobs=8, backend='multiprocessing', verbose=10)(\
                                  delayed(geode.compute_chdir_signature)(sig_id, row['distil_id'].split('|'), 
                                                                         distil_ids_sub, mat, mat_centered)\
                                  for sig_id, row in sig_info_df_sub.iterrows())
    # Write the part of the level5 matrix into the gctx file
    for sig_name, gctx5_mat in gctx5_mats.items():
        submat = np.asarray([doc[sig_name] for doc in docs], dtype=np.float32, order='F').T
        j1 = submat.shape[1]
        gctx5_mat.write_direct(submat, dest_sel=np.s_[:, j:(j+j1)]) 
    
    j += j1

===== Batch 0/75 AML001_PC3_6H =====
Number of signatures: 161
Number of pert samples: 506
(506, 978)


[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    3.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    6.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    7.7s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   12.3s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   16.8s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:   20.2s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   26.1s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:   31.5s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:   38.0s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:   44.5s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:   51.5s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:   58.6s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done 161 out of 161 | elapsed:  1.2min finished


===== Batch 1/75 ASG001_PC3_24H =====
Number of signatures: 84
Number of pert samples: 1000
(1000, 978)


[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    9.1s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   17.6s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   23.7s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   37.5s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   49.8s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done  78 out of  84 | elapsed:  1.8min remaining:    8.1s
[Parallel(n_jobs=8)]: Done  84 out of  84 | elapsed:  1.8min finished


===== Batch 2/75 ASG001_PC3_6H =====
Number of signatures: 84
Number of pert samples: 656
(656, 978)


...

[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    2.6s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    3.9s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    5.0s
[Parallel(n_jobs=8)]: Done  46 out of  55 | elapsed:    7.0s remaining:    1.4s
[Parallel(n_jobs=8)]: Done  52 out of  55 | elapsed:    7.6s remaining:    0.4s
[Parallel(n_jobs=8)]: Done  55 out of  55 | elapsed:    7.7s finished


===== Batch 73/75 PRISM001_PC3_6H =====
Number of signatures: 55
Number of pert samples: 226
(226, 978)


[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    2.4s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    3.5s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    4.5s
[Parallel(n_jobs=8)]: Done  46 out of  55 | elapsed:    6.3s remaining:    1.2s
[Parallel(n_jobs=8)]: Done  52 out of  55 | elapsed:    6.8s remaining:    0.4s
[Parallel(n_jobs=8)]: Done  55 out of  55 | elapsed:    6.9s finished


===== Batch 74/75 RAD001_PC3_6H =====
Number of signatures: 245
Number of pert samples: 500
(500, 978)


[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    2.7s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    5.4s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    7.7s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   11.6s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   15.1s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:   19.0s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   24.2s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:   29.8s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:   35.2s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:   41.5s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:   47.9s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:   55.3s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  1.3min

In [17]:
for gctx5_file in gctx5_files.values():
    gctx5_file.close()